Study the relationship between the velocity width of HI detections and the distance to the nearest neighbor for MaNGA galaxies.

In [1]:
from astropy.table import Table
import numpy as np
from scipy.spatial import KDTree

# Data

In [2]:
mangaHI_filename = 'input/mangaHIall.fits'
alfaHI_filename = 'input/manga_dr15_alfalfa.fits'
manga_filename =  'input/Pipe3D-master_file_vflag_10_smooth2p27_N2O2_noWords.txt'
#kias7_filename =  'kias1033_5_P-MJD-F_MPAJHU_ZdustOS_stellarMass_BPT_SFR_NSA_vflag.txt'
kias7_filename =  'input/kias1033_5_P-MJD-F_MPAJHU_ZdustOS_stellarMass_BPT_SFR_NSA_vflag.txt'

mangaHI = Table.read(mangaHI_filename, format='fits')
alfaHI = Table.read(alfaHI_filename, format='fits')
manga = Table.read(manga_filename, format='ascii.commented_header')
kias7 = Table.read(kias7_filename, format='ascii.commented_header')

## Add HI data to MaNGA table

##### Initialize new columns in MaNGA data table

In [3]:
manga['WF50'] = np.NaN*np.ones(len(manga), dtype=float)
manga['WP20'] = np.NaN*np.ones(len(manga), dtype=float)
manga['logHI'] = np.NaN*np.ones(len(manga), dtype=float)

##### Add ALFALFA HI data

In [4]:
for i in range(len(alfaHI)):
    
    galID = alfaHI['PLATEIFU'][i]
    plate, IFU = galID.split('-')
    
    # Find the corresponding row index for this galaxy in the MaNGA data table
    plate_boolean = manga['MaNGA_plate'] == int(plate)
    IFU_boolean = manga['MaNGA_IFU'] == int(IFU)
    manga_boolean = np.logical_and(plate_boolean, IFU_boolean)
    
    # Add data to MaNGA data table
    manga['WF50'][manga_boolean] = alfaHI['WF50'][i]
    manga['WP20'][manga_boolean] = alfaHI['WP20'][i]
    manga['logHI'][manga_boolean] = alfaHI['LOGMHI'][i]

##### Add GBT HI data

In [5]:
for i in range(len(mangaHI)):
    
    galID = mangaHI['plateifu'][i]
    plate, IFU = galID.split('-')
    
    # Find the corresponding row index for this galaxy in the MaNGA data table
    plate_boolean = manga['MaNGA_plate'] == int(plate)
    IFU_boolean = manga['MaNGA_IFU'] == int(IFU)
    manga_boolean = np.logical_and(plate_boolean, IFU_boolean)
    
    # Add data to MaNGA data table
    manga['WF50'][manga_boolean] = mangaHI['WF50'][i]
    manga['WP20'][manga_boolean] = mangaHI['WP20'][i]
    manga['logHI'][manga_boolean] = mangaHI['logMHI'][i]

## Isolate those MaNGA galaxies with HI detections

We only want to analyze those galaxies in MaNGA with HI detections, so let's create a subset of the main MaNGA data table that contains only those galaxies with HI detections.

In [6]:
HI_boolean = np.logical_or.reduce((np.isfinite(manga['WF50']), 
                                   np.isfinite(manga['WP20']), 
                                   np.isfinite(manga['logHI'])))

manga_HI = manga[HI_boolean]

In [7]:
manga_HI.write('input/gbt_alfalfa_HIdata.txt', format = 'ascii.commented_header', overwrite = True)

In [8]:
manga_HI

MaNGA_plate,MaNGA_IFU,NSA_plate,NSA_fiberID,NSA_MJD,NSA_index,NSA_RA,NSA_DEC,NSA_ba,NSA_phi,NSA_redshift,NSA_Mstar,vflag,avg_v_max,avg_v_max_sigma,avg_alpha,avg_alpha_sigma,avg_chi_square_rot,pos_v_max,pos_v_max_sigma,pos_alpha,pos_alpha_sigma,pos_chi_square_rot,neg_v_max,neg_v_max_sigma,neg_alpha,neg_alpha_sigma,neg_chi_square_rot,center_flux,center_flux_error,Mtot,Mtot_error,Mdark,Mdark_error,Mstar,Mdark_Mstar_ratio,Mdark_Mstar_ratio_error,rabsmag,avg_r_turn,avg_r_turn_sigma,pos_r_turn,pos_r_turn_sigma,neg_r_turn,neg_r_turn_sigma,logNO,t3,BPT,curve_used,points_cut,avg_chi_square_ndf,pos_chi_square_ndf,neg_chi_square_ndf,index,Z12logOH,u_r,frac_masked_spaxels,Rmax,Mtot_Mstar_ratio,Mtot_Mstar_ratio_error,smoothness_score,CMD_class,Mstar_turn,SFR,sSFR,pos_Vmax_data,avg_Vmax_data,neg_Vmax_data,WF50,WP20,logHI
int32,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64
8465,1901,1281,231,52753,247928,197.44581468983537,48.899591403437014,0.7684928178787231,18.52703857421875,0.028888285160064697,15041360896.0,0.0,694.1065598029447,64.28960377376522,169.88497626173015,1.7327077246997473e-05,15.217068463480258,670.7653939526836,15.960128227576048,6.971553117105116,3.3409887210988973,16.662168571733133,1824.214984439412,1128158.8728166134,10.139927187019886,342203353.66120255,562.2702918190502,0.2404831498861313,0.00282066542648091,235168173081.6036,43563538091.837006,212601943793.92502,43563538091.837006,22566229287.678577,9.421243623984985,1.9304748496738533,-19.729339599609375,1.543808157566983,0.16833871490329255,0.7834103370121185,0.04388640897002279,12.055743708240605,7457.1345332868905,-99.0,-99.0,2.0,-99,0,7.608534231740129,8.331084285866567,281.1351459095251,892095,9.46977439171353,2.427546977996826,0.07550335570469799,2.0993620223274516,10.421243623984985,1.9304748496738533,2.5540066976945326,3,-1.0,-1.0557212829589844,-11.407499313354492,835.9509258900858,705.7810819757563,-10.044590650342819,236.0,268.0,9.34000015258789
8249,3702,899,202,52620,164473,137.03265262994884,45.92096195150727,0.4280976951122284,47.86474609375,0.02679748833179474,4316812800.0,0.0,470.17155751503515,1764.5963567538108,0.9114709790034429,1.632624280324924,1.0792643567173899,34.92250964449018,1.714344719842902,137.1392992341442,579174444.6396726,3.3911074849327174,242.99019897544017,3887704333.4850802,171.2340152243181,0.0,26.156873020790904,0.08108656108379364,0.00201356164064311,180170543139.84866,1352392670031.6855,165985843226.1657,1352392670031.6855,14184699913.682957,11.701752186244782,95.34164827323065,-18.68962860107422,13.156762781102165,45.537164179135054,1.3032636040779486,0.18480939543790068,5.718307576497459,91489655.31072567,-99.0,-99.0,1.0,0,0,0.17987739278623163,0.5651845808221195,4.359478836798484,831037,8.984175177316411,2.073540985584259,0.07431796801505174,3.5053563122378297,12.701752186244782,95.34164827323065,1.481725305076537,2,8199197485.337716,0.0772257074713707,-9.959590911865234,39.338789135570636,90.5359425921002,-10.491716704738925,640.0,330.0,9.369999885559082
8247,6103,1200,480,52668,229715,136.71998244421826,41.408251709516264,0.6868849992752075,61.6627197265625,0.02735770121216774,40276209664.0,0.0,480.5235751703897,116.65286524888765,1.383578972545609,0.6374423878026014,32.27335181919168,407.07638916038286,17.12196986199509,4.900489396452084,2.180848875031777,205.39409595676938,278.71867981063366,40.72662777376588,29.059133229461185,634.7538393034314,212.15424466509577,0.2653121054172516,0.00239786966525532,234820585972.89932,114011031374.66031

In [9]:
np.log10(15041360896)

10.177287131660222

# Find distance to nearest neighbor for each MaNGA galaxy

For each galaxy in the MaNGA data table, find its nearest neighbor and calculate the distance to that galaxy.
1. Calculate the distance to every galaxy in SDSS DR7 for each MaNGA galaxy with an HI detection.
2. Find the galaxy with the smallest distance - this is our nearest neighbor!  (Note: We need to check to make sure that we do not identify the MaNGA HI target galaxy as the nearest neighbor.)

### Convert (ra, dec, redshift) to (x, y, z)

Since we are going to do this for both the KIAS galaxies and the MaNGA galaxies, let's write a function.

In [10]:
c = 3e5 #3*(10**8) m/s <-- Scientific notation
H = 100 #2.43 * (10**-18) 1/s <-- We are doing to do all of our distances in Mpc/h (where H0 = 100h).

def rdz_to_xyz(data):
    '''
    Convert ra, dec, redshift to x, y, z
    
    
    PARAMETERS
    ==========
    
    data : astropy table
        Galaxies to analyze.  Columns include ra, dec, redshift.
        
        
    RETURNS
    =======
    
    data : astropy table
        Same as input data object, with the additional fields of x, y, z
    '''

    '''
    # We can do this all at once, since we are doing the same calculation on every 
    # row of the data table.
    
    for i in range(len(data)):
        radius_2 = c*data['redshift']/H
        r_2 = radius_2
        x_2_cor = r_2 * np.cos(kias7['ra'][i]* np.pi/180) * np.cos(kias7['dec'][i]* np.pi/180)
        y_2_cor = r_2 * np.sin(kias7['ra'][i]* np.pi/180) * np.cos(kias7['dec'][i]* np.pi/180)
        z_2_cor = r_2 * np.sin(kias7['ra'][i]* np.pi/180)
        x_2.append(x_2_cor)
        y_2.append(y_2_cor)
        z_2.append(z_2_cor)
    '''
    
    ################################################################################
    # Our data tables have different column names, so we need to determine the 
    # correct names.
    #-------------------------------------------------------------------------------
    if 'redshift' in data.colnames:
        redshift = data['redshift']
        ra = data['ra'] * np.pi/180
        dec = data['dec'] * np.pi/180
    elif 'NSA_redshift' in data.colnames:
        redshift = data['NSA_redshift']
        ra = data['NSA_RA'] * np.pi/180
        dec = data['NSA_DEC'] * np.pi/180
    else:
        print('Unknown field names.  Add to rdz_to_xyz function and rerun.')
        return data
    ################################################################################
    
    
    ################################################################################
    # Convert from spherical to cartesian coordinates
    #-------------------------------------------------------------------------------
    r = c*redshift/H
    
    data['x'] = r * np.cos(ra) * np.cos(dec)
    data['y'] = r * np.sin(ra) * np.cos(dec)
    data['z'] = r * np.sin(dec)
    ################################################################################
    
    return data

In [11]:
# Now that we have our function written, we can do this conversion on both data tables
manga_HI = rdz_to_xyz(manga_HI)
kias7 = rdz_to_xyz(kias7)

### Calculate distance between galaxies and find the closest one

In [12]:
# Generates x_tree, y_tree, z_tree array for kias7
x_tree = np.asarray(kias7['x'])
x_tree = x_tree.astype(np.float64)
x_tree = x_tree.reshape(x_tree.shape[0],1)
y_tree = np.asarray(kias7['y'])
y_tree = y_tree.astype(np.float64)
y_tree = y_tree.reshape(y_tree.shape[0],1)
z_tree = np.asarray(kias7['z'])
z_tree = z_tree.astype(np.float64)
z_tree = z_tree.reshape(z_tree.shape[0],1)
xyz_tree = np.concatenate((x_tree,y_tree,z_tree), axis=1)

# Generates x_query, y_query, z_query array for manga
x_query = np.asarray(manga_HI['x'])
x_query = x_query.astype(np.float64)
x_query = x_query.reshape(x_query.shape[0],1)
y_query = np.asarray(manga_HI['y'])
y_query = y_query.astype(np.float64)
y_query = y_query.reshape(y_query.shape[0],1)
z_query = np.asarray(manga_HI['z'])
z_query = z_query.astype(np.float64)
z_query = z_query.reshape(z_query.shape[0],1)
xyz_query = np.concatenate((x_query,y_query,z_query), axis=1)

In [13]:
# Generate KDTree for kias7
tree = KDTree(xyz_tree)

In [14]:
nearest_distances, _ = tree.query(xyz_query, k=2)
nearest_distances = nearest_distances[:,1]
nearest_distances

array([1.23878972, 0.657584  , 0.35897515, ..., 0.2109972 , 0.47263304,
       0.87133691])

In [15]:
manga_HI['d_nn'] = np.zeros(len(manga_HI), dtype=float)
manga_HI['d_nn'] = nearest_distances

In [16]:
manga_HI

MaNGA_plate,MaNGA_IFU,NSA_plate,NSA_fiberID,NSA_MJD,NSA_index,NSA_RA,NSA_DEC,NSA_ba,NSA_phi,NSA_redshift,NSA_Mstar,vflag,avg_v_max,avg_v_max_sigma,avg_alpha,avg_alpha_sigma,avg_chi_square_rot,pos_v_max,pos_v_max_sigma,pos_alpha,pos_alpha_sigma,pos_chi_square_rot,neg_v_max,neg_v_max_sigma,neg_alpha,neg_alpha_sigma,neg_chi_square_rot,center_flux,center_flux_error,Mtot,Mtot_error,Mdark,Mdark_error,Mstar,Mdark_Mstar_ratio,Mdark_Mstar_ratio_error,rabsmag,avg_r_turn,avg_r_turn_sigma,pos_r_turn,pos_r_turn_sigma,neg_r_turn,neg_r_turn_sigma,logNO,t3,BPT,curve_used,points_cut,avg_chi_square_ndf,pos_chi_square_ndf,neg_chi_square_ndf,index,Z12logOH,u_r,frac_masked_spaxels,Rmax,Mtot_Mstar_ratio,Mtot_Mstar_ratio_error,smoothness_score,CMD_class,Mstar_turn,SFR,sSFR,pos_Vmax_data,avg_Vmax_data,neg_Vmax_data,WF50,WP20,logHI,x,y,z,d_nn
int32,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
8465,1901,1281,231,52753,247928,197.44581468983537,48.899591403437014,0.7684928178787231,18.52703857421875,0.028888285160064697,15041360896.0,0.0,694.1065598029447,64.28960377376522,169.88497626173015,1.7327077246997473e-05,15.217068463480258,670.7653939526836,15.960128227576048,6.971553117105116,3.3409887210988973,16.662168571733133,1824.214984439412,1128158.8728166134,10.139927187019886,342203353.66120255,562.2702918190502,0.2404831498861313,0.00282066542648091,235168173081.6036,43563538091.837006,212601943793.92502,43563538091.837006,22566229287.678577,9.421243623984985,1.9304748496738533,-19.729339599609375,1.543808157566983,0.16833871490329255,0.7834103370121185,0.04388640897002279,12.055743708240605,7457.1345332868905,-99.0,-99.0,2.0,-99,0,7.608534231740129,8.331084285866567,281.1351459095251,892095,9.46977439171353,2.427546977996826,0.07550335570469799,2.0993620223274516,10.421243623984985,1.9304748496738533,2.5540066976945326,3,-1.0,-1.0557212829589844,-11.407499313354492,835.9509258900858,705.7810819757563,-10.044590650342819,236.0,268.0,9.34000015258789,-54.35114534553452,-17.08035669799775,65.30705620452258,1.2387897185381238
8249,3702,899,202,52620,164473,137.03265262994884,45.92096195150727,0.4280976951122284,47.86474609375,0.02679748833179474,4316812800.0,0.0,470.17155751503515,1764.5963567538108,0.9114709790034429,1.632624280324924,1.0792643567173899,34.92250964449018,1.714344719842902,137.1392992341442,579174444.6396726,3.3911074849327174,242.99019897544017,3887704333.4850802,171.2340152243181,0.0,26.156873020790904,0.08108656108379364,0.00201356164064311,180170543139.84866,1352392670031.6855,165985843226.1657,1352392670031.6855,14184699913.682957,11.701752186244782,95.34164827323065,-18.68962860107422,13.156762781102165,45.537164179135054,1.3032636040779486,0.18480939543790068,5.718307576497459,91489655.31072567,-99.0,-99.0,1.0,0,0,0.17987739278623163,0.5651845808221195,4.359478836798484,831037,8.984175177316411,2.073540985584259,0.07431796801505174,3.5053563122378297,12.701752186244782,95.34164827323065,1.481725305076537,2,8199197485.337716,0.0772257074713707,-9.959590911865234,39.338789135570636,90.5359425921002,-10.491716704738925,640.0,330.0,9.369999885559082,-40.92269992292892,38.11745635132102,57.752407571954315,0.657583997773578
8247,6103,1200,480,52668,229715,136.71998244421826,41.408251709516264,0.6868849992752075,61.6627197265625,0.02735770121216774,40276209664.0,0.0,480.5235751703897,116.65286524888765,1.383578972545609,0.6374423878026014,32.27335181919168,407.07638916038286,17.12196986199509,4.900489396452084,2.180848875031